Load modules

In [ ]:
using Plots
using NCDatasets
using Statistics: mean

Setup run-time enviromnent

In [ ]:
ENV["GKSwstype"] = "100"

Specify needed directories and filenames: these will be replaced automatically by the bash run script or can be modified manually

In [ ]:
CLIMA_NETCDF = "/central/scratch/elencz/caltech_fms_idealized/hs_syn_diff/hs_syn_diff_num_fourier_42_lat0_s_0.0/output/ppp_output/history/";

Get the current and previous  GCM netcdf file names in the CLIMA_NETCDF directory

In [ ]:
#file_no = 50
#fnames = filter(x -> occursin(".nc", x), readdir( CLIMA_NETCDF ) );
#filename = "$CLIMA_NETCDF"fnames[file_no] # selects the last file on list
#plot_dir = fnames[file_no]*"_plots/"
#mkdir(plot_dir)

#println(size(fnames))
#println(filename)

extract data

In [ ]:
#ds = NCDataset(filename, "r");
#lon = ds["lon"][:];
#lat = ds["lat"][:];
#lev = ds["sigma"][:] ; # sigma coord
#u = ds["u"][:];
#v = ds["v"][:];
#T = ds["streamfctn"][:];
#close(ds)

#size(T)

In [ ]:
function extract_snapshot(filename, time)
    ds = NCDataset(filename*"$time.nc", "r");
    lon = ds["lon"][:];
    lat = ds["lat"][:];
    lev = ds["sigma"][:] ; # sigma coord
    u = ds["u"][:];
    v = ds["v"][:];
    T = ds["streamfctn"][:];
    close(ds)
    return lon,lat,lev,u,v,T
end
    
function extract_timemean(filename, times)
    u = Any[]
    v = Any[]
    T = Any[]
    ct = 0
    for time in times
        ct += 1
        ds = NCDataset(filename*"$time.nc", "r");
        lon = ds["lon"][:];
        lat = ds["lat"][:];
        lev = ds["sigma"][:] ; # sigma coord
        if ct==1
            u = ds["u"][:] ./ length(times);
            v = ds["v"][:] ./ length(times);
            T = ds["streamfctn"][:] ./ length(times);
        else             
            u += ds["u"][:] ./ length(times);
            v += ds["v"][:] ./ length(times);
            T += ds["streamfctn"][:] ./ length(times);
        end
        close(ds)
    end
    return lon,lat,lev,u,v,T
end


fourier_no = 3
lat0 = 6
CLIMA_NETCDF = string("/central/scratch/elencz/caltech_fms_idealized/hs_syn_diff/hs_syn_diff_num_fourier_",fourier_no,"_lat0_s_",lat0,".0/output/ppp_output/history/");

plot_dir = string("plots/","yr5_num_fourier_",fourier_no,"_lat0_s_",lat0)

fname_root = "day1800h00.segment"
filename = "$CLIMA_NETCDF"*fname_root

lon,lat,lev,u_0,v_0,T_0 = extract_snapshot(filename, 100)

times=collect(1:1:360)
lon,lat,lev,u,v,T = extract_timemean(filename, times)



In [ ]:
size(T)


In [ ]:
# zonal mean:T and u at last diagnostic time

u_zm = mean( u[:,:,:,:], dims=1)[1,:,:,:]; # lon, lat,lev
T_zm = mean( T[:,:,:,:], dims=1)[1,:,:,:]; # lon, lat,lev 
v_zm = sqrt.(mean( v[:,:,:] .^ 2, dims=1)[1,:,:]); # lon, lat,lev

plot1 = contourf( lat, lev, (u_zm[:,:] )', title="u", xlabel="lat (deg N)", ylabel="sigma", linewidth = 0, yflip=true);
plot1 = contourf( lat, lev, (0.5 .* (u_zm[:,:] + u_zm[end:-1:1,:]) )', title="u", xlabel="lat (deg N)", ylabel="sigma", linewidth = 0, yflip=true);
plot_array = [plot1]
plot2 = contourf( lat, lev, (T_zm[:,:] ./ 1e9)', title="streamfunction", xlabel="lat (deg N)", ylabel="sigma", linewidth = 0, yflip=true);
plot2 = contourf( lat, lev, (0.5 .* (T_zm[:,:] - T_zm[end:-1:1,:])./ 1e9 )', title="streamfunction", xlabel="lat (deg N)", ylabel="sigma", linewidth = 0, yflip=true);
push!(plot_array,plot2);
plot3 = contourf( lat, lev, (v_zm[:,:])', title="sqrt(v^2)", xlabel="lat (deg N)", ylabel="sigma", linewidth = 0, yflip=true);
plot3 = contourf( lat, lev, (0.5 .* (v_zm[:,:] + v_zm[end:-1:1,:]) )', title="sqrt(v^2)", xlabel="lat (deg N)", ylabel="sigma", linewidth = 0, yflip=true);
push!(plot_array,plot3);
fig=plot(plot_array... , layout=(1, 3), size=(1200, 200) )
#savefig(fig, plot_dir*"zonal_mean.pdf");
savefig(fig, plot_dir*"zonal_mean_sum.pdf");
display(fig)

vertical slice of v at lev_index

In [ ]:
lev_index = 5
z_in_km = lev[lev_index]
v_plot = contourf( lon, lat, (v_0[:,:,lev_index])', title="v @ $z_in_km", xlabel="lon (deg)", ylabel="lat (deg N)", linewidth = 0);
fig=plot(v_plot, layout=(1, 1), size=(800, 400) )
savefig(fig, plot_dir*"vertical_slice.pdf");
display(fig)

In [ ]:
# Animation
clims = (-10,10)
diag_dt_days =  (time[2] - time[1]).value / (1000*60*60*24) # get simtime

lev_index_tropos = 10
lev_tropos = lev[lev_index_tropos]

anim = @animate for t_i in 1:length(time)
    plot_array = []
    plot_zm = contourf( lat, lev, (v_zm[:,:,t_i])', title="sqrt(v^2)", xlabel="lat (deg N)", ylabel="z (km)", clims = clims, linewidth=0);
    push!(plot_array,plot_zm); 
    plot_h = contourf( lon, lat, (v[:,:,lev_index_tropos,t_i])', title="v @ $lev_tropos km", xlabel="lon (deg)", ylabel="lat (deg N)", clims = clims, linewidth=0);
    push!(plot_array,plot_h);  
    time_=time[t_i]
    plot_h = contourf( lon, lat, (v[:,:,lev_index,t_i])', title="v @ $z_in_km km @ $time_ s", xlabel="lon (deg)", ylabel="lat (deg N)", clims = clims, linewidth=0);
    push!(plot_array,plot_h);  
    plot(plot_array..., layout=(1,3), size=(1200, 400) ) 
end
mp4(anim, plot_dir*"plot_y_slice_anim.gif", fps = 7) # hide, mp4 is more flaky

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*

In [ ]:
time[end]